In [ ]:
#@markdown Check type of GPU and VRAM available.
!nvidia-smi --query-gpu=name,memory.total,memory.free --format=csv,noheader

/bin/bash: line 1: nvidia-smi: command not found


# Install Dependencies

In [ ]:
!mkdir data
!mkdir data/diagramatic-architecture

In [ ]:
!git clone https://github.com/huggingface/diffusers.git
%pip install -qq git+https://github.com/huggingface/diffusers

Cloning into 'diffusers'...
remote: Enumerating objects: 41359, done.
remote: Counting objects: 100% (1157/1157), done.
remote: Compressing objects: 100% (573/573), done.
remote: Total 41359 (delta 771), reused 794 (delta 490), pack-reused 40202
Receiving objects: 100% (41359/41359), 27.49 MiB | 30.23 MiB/s, done.
Resolving deltas: 100% (30577/30577), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 37.2 MB/s eta 0:00:00


In [ ]:
%pip install --upgrade pip

In [ ]:
%pip install -q -U --pre triton
%pip install -q accelerate transformers ftfy bitsandbytes==0.35.0 gradio natsort safetensors
%pip install -U xformers

In [ ]:
%cd diffusers/examples/

/content/diffusers/examples


## Hugging Face Credentials

In [ ]:
#@title Login to HuggingFace 🤗

#@markdown You need to accept the model license before downloading or using the Stable Diffusion weights. Please, visit the [model card](https://huggingface.co/runwayml/stable-diffusion-v1-5), read the license and tick the checkbox if you agree. You have to be a registered user in 🤗 Hugging Face Hub, and you'll also need to use an access token for the code to work.
# https://huggingface.co/settings/tokens
!mkdir -p ~/.huggingface
HUGGINGFACE_TOKEN = "" #@param {type:"string"}
!echo -n "{HUGGINGFACE_TOKEN}" > ~/.huggingface/token

# Settings

In [ ]:
#@markdown If model weights should be saved directly in google drive (takes around 4-5 GB).
save_to_gdrive = True #@param {type:"boolean"}
if save_to_gdrive:
    from google.colab import drive
    drive.mount('/content/drive', force_remount=True)

#@markdown Name/Path of the initial model.
MODEL_NAME = "runwayml/stable-diffusion-v1-5" #@param {type:"string"}

#@markdown Enter the directory name to save model at.

OUTPUT_DIR = "weights" #@param {type:"string"}
if save_to_gdrive:
    OUTPUT_DIR = "/content/drive/MyDrive/Personal/Coding/Github\ Projects/diagramatic-architecture/" + OUTPUT_DIR
else:
    OUTPUT_DIR = "/content/" + OUTPUT_DIR

print(f"[*] Weights will be saved at {OUTPUT_DIR}")

!mkdir -p $OUTPUT_DIR

Mounted at /content/drive
[*] Weights will be saved at /content/drive/MyDrive/Personal/Coding/Github\ Projects/diagramatic-architecture/weights


# Training

Use the table below to choose the best flags based on your memory and speed requirements. Tested on Tesla T4 GPU.


| `fp16` | `train_batch_size` | `gradient_accumulation_steps` | `gradient_checkpointing` | `use_8bit_adam` | GB VRAM usage | Speed (it/s) |
| ---- | ------------------ | ----------------------------- | ----------------------- | --------------- | ---------- | ------------ |
| fp16 | 1                  | 1                             | TRUE                    | TRUE            | 9.92       | 0.93         |
| no   | 1                  | 1                             | TRUE                    | TRUE            | 10.08      | 0.42         |
| fp16 | 2                  | 1                             | TRUE                    | TRUE            | 10.4       | 0.66         |
| fp16 | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 1.14         |
| no   | 1                  | 1                             | FALSE                   | TRUE            | 11.17      | 0.49         |
| fp16 | 1                  | 2                             | TRUE                    | TRUE            | 11.56      | 1            |
| fp16 | 2                  | 1                             | FALSE                   | TRUE            | 13.67      | 0.82         |
| fp16 | 1                  | 2                             | FALSE                   | TRUE            | 13.7       | 0.83          |
| fp16 | 1                  | 1                             | TRUE                    | FALSE           | 15.79      | 0.77         |
*italicized text*

Add `--gradient_checkpointing` flag for around 9.92 GB VRAM usage.

remove `--use_8bit_adam` flag for full precision. Requires 15.79 GB with `--gradient_checkpointing` else 17.8 GB.

remove `--train_text_encoder` flag to reduce memory usage further, degrades output quality.

In [ ]:
!python3 dreambooth/train_dreambooth.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --output_dir=$OUTPUT_DIR \
  --revision="fp16" \
  --with_prior_preservation --prior_loss_weight=1.0 \
  --seed=1337 \
  --resolution=512 \
  --train_batch_size=1 \
  --train_text_encoder \
  --mixed_precision="fp16" \
  --gradient_checkpointing \
  --use_8bit_adam \
  --gradient_accumulation_steps=1 \
  --learning_rate=1e-6 \
  --lr_scheduler="constant" \
  --lr_warmup_steps=0 \
  --num_class_images=50 \
  --sample_batch_size=4 \
  --max_train_steps=1000 \
  --instance_data_dir="/content/data/diagramatic-architecture" \
  --instance_prompt="photo of diagramatic architecture" \
  --class_data_dir="/content/data/diagram" \
  --class_prompt="photo of a diagram"

# Reduce the `--save_interval` to lower than `--max_train_steps` to save weights from intermediate steps.
# `--save_sample_prompt` can be same as `--instance_prompt` to generate intermediate samples (saved along with weights in samples directory).

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.1.0+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.13 (you have 3.10.12)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
2023-10-27 04:08:26.484788: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-27 04:08:26.484839: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-27 04:08:26.484873: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin c

#Export

In [ ]:
#@markdown Run conversion.
WEIGHTS_DIR = "contents/drive/MyDrive/Personal/Coding/Github Projects/diagramatic-architecture/weights"
ckpt_path = "contents/drive/MyDrive/Personal/Coding/Github Projects/diagramatic-architecture/weights/model.ckpt"

half_arg = ""
#@markdown  Whether to convert to fp16, takes half the space (2GB).
fp16 = True #@param {type: "boolean"}
if fp16:
    half_arg = "--half"
!python ../scripts/convert_diffusers_to_original_stable_diffusion.py --model_path $WEIGHTS_DIR  --checkpoint_path $ckpt_path $half_arg
print(f"[*] Converted ckpt saved at {ckpt_path}")

usage: convert_diffusers_to_original_stable_diffusion.py [-h] --model_path MODEL_PATH
                                                         --checkpoint_path CHECKPOINT_PATH
                                                         [--half] [--use_safetensors]
convert_diffusers_to_original_stable_diffusion.py: error: unrecognized arguments: Projects/diagramatic-architecture/weights Projects/diagramatic-architecture/weights/model.ckpt
[*] Converted ckpt saved at contents/drive/MyDrive/Personal/Coding/Github Projects/diagramatic-architecture/weights/model.ckpt


In [15]:
WEIGHTS_DIR = "/content/drive/MyDrive/Personal/Coding/Github Projects/diagramatic-architecture/weights"
ckpt_path = "/content/drive/MyDrive/Personal/Coding/Github Projects/diagramatic-architecture/weights/model.ckpt"

half_arg = ""
fp16 = True
if fp16:
    half_arg = "--half"

cmd = f"python ../scripts/convert_diffusers_to_original_stable_diffusion.py --model_path '{WEIGHTS_DIR}' --checkpoint_path '{ckpt_path}' {half_arg}"
!{cmd}
print(f"[*] Converted ckpt saved at {ckpt_path}")


Traceback (most recent call last):
  File "/content/diffusers/examples/../scripts/convert_diffusers_to_original_stable_diffusion.py", line 290, in <module>
    unet_state_dict = torch.load(unet_path, map_location="cpu")
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 986, in load
    with _open_file_like(f, 'rb') as opened_file:
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 435, in _open_file_like
    return _open_file(name_or_buffer, mode)
  File "/usr/local/lib/python3.10/dist-packages/torch/serialization.py", line 416, in __init__
    super().__init__(open(name, mode))
FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Personal/Coding/Github Projects/diagramatic-architecture/weights/unet/diffusion_pytorch_model.bin'
[*] Converted ckpt saved at /content/drive/MyDrive/Personal/Coding/Github Projects/diagramatic-architecture/weights/model.ckpt
